In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import scipy.stats as st
import matplotlib.pyplot as plt
import time
import statsmodels.api as sm
client=pd.read_csv("Data/customers.csv")
produit=pd.read_csv("Data/products.csv")
transaction=pd.read_csv("Data/transactions.csv")


# Nettoyage de Produit

In [4]:
produit.head(n=3)

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0


In [5]:
produit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
id_prod    3287 non-null object
price      3287 non-null float64
categ      3287 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 77.1+ KB


In [6]:
produit=pd.read_csv("products.csv",dtype={'categ':'category'})

In [7]:
produit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
id_prod    3287 non-null object
price      3287 non-null float64
categ      3287 non-null category
dtypes: category(1), float64(1), object(1)
memory usage: 54.7+ KB


In [8]:
produit.describe() #Ici on observe qu'il y a un price négatif chose impossible

,price
count,3287.000000
mean,21.856641
std,29.847908
min,-1.000000
25%,6.990000
50%,13.060000
75%,22.990000
max,300.000000


In [9]:
print(produit.loc[produit['price']<0,:])#On va identifier ce prix négatif rapidement 

    id_prod  price categ
731     T_0   -1.0     0


In [10]:
produit= produit[~produit['id_prod'].str.startswith('T_')]#on s'apercois que l'id commence par 'T_'chose 
#inhabituelle ds la structure de l'id, donc on va le retirer 
produit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3286 entries, 0 to 3286
Data columns (total 3 columns):
id_prod    3286 non-null object
price      3286 non-null float64
categ      3286 non-null category
dtypes: category(1), float64(1), object(1)
memory usage: 80.2+ KB


 # Nettoyage de de transaction 

In [11]:
transaction.head(n=3)

,id_prod,date,session_id,client_id
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270


In [12]:
transaction= transaction[~transaction['id_prod'].str.startswith('T_')]#On nettoie du coup partout cet id 

In [13]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336816 entries, 0 to 337015
Data columns (total 4 columns):
id_prod       336816 non-null object
date          336816 non-null object
session_id    336816 non-null object
client_id     336816 non-null object
dtypes: object(4)
memory usage: 12.8+ MB


# Nettoyage de client 

In [14]:
client.head(n=3)

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984


In [15]:
client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
client_id    8623 non-null object
sex          8623 non-null object
birth        8623 non-null int64
dtypes: int64(1), object(2)
memory usage: 202.2+ KB


In [16]:
client=pd.read_csv("Data/customers.csv",dtype={'sex':'category'})
#Le but est de transformer la variable sexe en catégorie 

In [17]:
client=client[~client['client_id'].str.startswith('ct_')]#ici on identifie le 'ct_' qui est l'équivalent du 'T_'
client.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8621 entries, 0 to 8622
Data columns (total 3 columns):
client_id    8621 non-null object
sex          8621 non-null category
birth        8621 non-null int64
dtypes: category(1), int64(1), object(1)
memory usage: 210.5+ KB


In [18]:
print((client['client_id']=="ct_0").value_counts())

False    8621
Name: client_id, dtype: int64


# FUSION DES DATASETS

In [19]:
Merge1=transaction.merge(produit, left_on='id_prod', right_on='id_prod', how='left')
#On réalise un merge à gauche pour verifier qu'on ne perd pas des produits avec un inner 

In [20]:
Merge1.head(n=4)

,id_prod,date,session_id,client_id,price,categ
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450,4.99,0
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277,65.75,2
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270,10.71,1
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597,4.20,0


In [21]:
sell=Merge1.merge(client,left_on='client_id',right_on='client_id',how='left') #On continue pour tout fusionner 

In [22]:
sell.head()

,id_prod,date,session_id,client_id,price,categ,sex,birth
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450,4.99,0,f,1977
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277,65.75,2,f,2000
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270,10.71,1,f,1979
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597,4.20,0,m,1963
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242,8.99,0,f,1980


In [23]:
sell.head(n=4)

,id_prod,date,session_id,client_id,price,categ,sex,birth
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450,4.99,0,f,1977
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277,65.75,2,f,2000
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270,10.71,1,f,1979
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597,4.20,0,m,1963


In [24]:
sell.date=pd.to_datetime(sell.date) #On transforme notre colonne date en type date

In [25]:
sell.head(n=4)

,id_prod,date,session_id,client_id,price,categ,sex,birth
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450,4.99,0,f,1977
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277,65.75,2,f,2000
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270,10.71,1,f,1979
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597,4.20,0,m,1963


In [26]:
sell.date[1]#On vérifie juste la modification 

Timestamp('2022-02-03 01:55:53.276402')

In [27]:
sell.info()
#Ici on affiche notre jeu de données pour verifer le nombre non null et observer des inconhérences 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336816 entries, 0 to 336815
Data columns (total 8 columns):
id_prod       336816 non-null object
date          336816 non-null datetime64[ns]
session_id    336816 non-null object
client_id     336816 non-null object
price         336713 non-null float64
categ         336713 non-null category
sex           336816 non-null category
birth         336816 non-null int64
dtypes: category(2), datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 28.6+ MB


In [28]:
print(sell.loc[sell['price'].isnull(),:])
#On vient d'identifier dans price 103 valeurs qui sont nulles 

       id_prod                       date session_id client_id  price categ  \
6231    0_2245 2021-06-17 03:03:12.668129    s_49705    c_1533    NaN   NaN   
10797   0_2245 2021-06-16 05:53:01.627491    s_49323    c_7954    NaN   NaN   
14045   0_2245 2021-11-24 17:35:59.911427   s_124474    c_5120    NaN   NaN   
17480   0_2245 2022-02-28 18:08:49.875709   s_172304    c_4964    NaN   NaN   
21071   0_2245 2021-03-01 00:09:29.301897        s_3     c_580    NaN   NaN   
21754   0_2245 2022-02-11 09:05:43.952857   s_163405    c_1098    NaN   NaN   
22672   0_2245 2021-10-19 00:28:01.920054   s_106841    c_3953    NaN   NaN   
24576   0_2245 2022-02-25 00:08:08.736068   s_170426    c_6236    NaN   NaN   
30874   0_2245 2021-08-22 08:51:27.564509    s_79102    c_6752    NaN   NaN   
31330   0_2245 2021-05-12 03:36:34.586221    s_33316    c_6205    NaN   NaN   
34893   0_2245 2021-10-15 09:31:31.539354   s_105069    c_4188    NaN   NaN   
35568   0_2245 2021-12-29 23:24:51.621799   s_142078

In [29]:
sell.loc[sell.isnull().any(axis=1)].shape

(103, 8)

A partir de la nous avons quatre choix pour traitrer nos NAN : 
1)Supprimer les lignes : si les NAN<5% du jeu de données 
2)Inputation par valeur fixe (on remplace en effectuant : moyenne, median etc...)
3)Inputation par méthode de prédiction, remplacer par ce qui aurait pu être 
4)Si les nb de NAN sont trop importants, et si les autres solutions ne marchent pas alors on les supprimer

Ici on aurait pu opter pour la première solution, en les supprimant, néanmoins mon tuteur m'a demandé de faire la faire la méthode numéro 2. 

In [30]:
sell.head(n=30)

,id_prod,date,session_id,client_id,price,categ,sex,birth
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450,4.99,0,f,1977
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277,65.75,2,f,2000
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270,10.71,1,f,1979
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597,4.20,0,m,1963
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242,8.99,0,f,1980
5,0_1085,2021-09-15 05:47:48.215162,s_90139,c_2526,3.99,0,f,1982
6,0_1508,2021-08-29 05:39:01.055455,s_82100,c_5799,8.03,0,f,1962
7,0_1627,2021-09-19 14:54:52.410362,s_92294,c_1422,3.99,0,f,1980
8,0_1469,2022-02-03 07:13:22.559995,s_159252,c_2207,14.99,0,f,1970
9,0_1453,2022-02-26 09:03:10.571665,s_171098,c_5433,7.99,0,f,1981


In [31]:
moyenne_cat0=sell.loc[sell['categ']=='0','price'].mean()
#Ici on s'aperçoit que le début de l'id production correspond en général à sa categorie de produit. 
#Ici on n'observe alors que nous avons que des produits de catégorie 0
#Ici nous faisons tout simplement la moyenne des produits de catégorie pour remplacer ensuite sur les NAN

In [32]:
sell.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336816 entries, 0 to 336815
Data columns (total 8 columns):
id_prod       336816 non-null object
date          336816 non-null datetime64[ns]
session_id    336816 non-null object
client_id     336816 non-null object
price         336713 non-null float64
categ         336713 non-null category
sex           336816 non-null category
birth         336816 non-null int64
dtypes: category(2), datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 28.6+ MB


In [33]:
print(moyenne_cat0)

10.646828235274288


In [34]:
sell.loc[sell['price'].isnull(), 'price'] = 10.646828235274288
sell.loc[sell['categ'].isnull(), 'categ']='0'
#Nous avons donc calculé la moyenne d'un produit de catégorie 0 et nous remplacons 
#Et comme nous savons que se sont des produits de categorie 0, on va l'écrire 

In [35]:
sell['age']=2022-sell['birth']
#Ici nous sommes intéressé pour connaître l'age de nos clients pour les missions à venir

In [36]:
sell.head(n=3)

,id_prod,date,session_id,client_id,price,categ,sex,birth,age
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450,4.99,0,f,1977,45
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277,65.75,2,f,2000,22
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270,10.71,1,f,1979,43


In [37]:
sell.to_csv("Data/sell.csv")